# Новосибирск

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline

In [81]:
def to_string(a):
    if isinstance(a, float):
        return str(int(a))
    elif isinstance(a, int):
        return str(a)
    else:
        return str(a)

DF = pd.DataFrame()    
   
for file in os.listdir('.'):
    if 'xlsx' in file and '$' not in file:
        dfs = pd.read_excel(file, sheet_name=None, dtype=str)
        for df in dfs:
            temp_df = dfs[df]     
            temp_df = temp_df[~(temp_df['Дата и время'].isin(['Июль', 'Август', 'Сентябрь']))]
            temp_df['Дата и время'] = pd.to_datetime(temp_df['Дата и время'])
            temp_df['Файл'] = [file] * temp_df.shape[0]
            temp_df['Кампания'] = [df] * temp_df.shape[0]
            DF = DF.append(temp_df)
            
DF = DF.fillna('')            
DF = DF[DF['Номер абонента'] != '']            
DF = DF.sort_values(by='Дата и время', ignore_index=True)
DF = DF.drop_duplicates(subset='Номер абонента', ignore_index=True) 


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [82]:
def join_list(lst):
    return ', '.join([str(i) for i in lst])



au_telefons = pd.read_excel(r'C:\Users\AmanovRA\Текущее2\РК_3_квартал\НСК\НСК_оборот\НСК_оборот.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)                 

lst_with_coincidenc_telefon  = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append('')
    for i in DF['Номер абонента']:
        if str(i) in row:
            lst_with_coincidenc_telefon[index] = str(i)
            break
            
au_telefons['Совпадение'] = lst_with_coincidenc_telefon            
            
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']
au_telefons_final = au_telefons_final.merge(DF, left_on='Совпадение', right_on='Номер абонента', how='left')
au_telefons_final = au_telefons_final.fillna('')


# не забыть проверить чтобы даты приемов были больше или равны дате звонка          



In [83]:
au_telefons_final['Совпадение дат'] = au_telefons_final['Дата приема'] >= au_telefons_final['Дата и время']

au_telefons_final = au_telefons_final[au_telefons_final['Совпадение дат'] == True]

au_telefons_final.to_excel('НСК_обработано.xlsx')